In [1]:
import keras

In [2]:
import tensorflow as tf

In [3]:
from tensorflow.keras.layers import Input,Embedding,GRU,Bidirectional,Dense

In [4]:
from tensorflow.keras import Model

In [5]:
import numpy as np

In [6]:
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)
        
    def call(self, inputs):
        query,values=inputs
        query_with_time_axis = tf.expand_dims(query, 1)
        score1=self.W1(query_with_time_axis)
        score2=self.W2(values)
        combined_score=tf.nn.tanh(score1 + score2)
        score = self.V(combined_score)
        attention_weights = tf.nn.softmax(score, axis=1)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector

In [34]:
encoder_inputs=Input(shape=(max_inp_len,))
encoder_embed=Embedding(src_vocab_size,embed_size)(encoder_inputs)
encoder_gru=Bidirectional(GRU(gru_units,return_sequences=True,return_state=True))
encoder_op,forward,backward=encoder_gru(encoder_embed)
encoder_dense=Dense(gru_units)
hidden=encoder_dense((tf.concat([forward,backward], axis = -1)))


decoder_inputs=Input(shape=(max_tar_len,))
decoder_embeded=Embedding(trg_vocab_size,embed_size)
decoder_embedded=decoder_embeded(decoder_inputs)
attention=BahdanauAttention(gru_units)
context_vector=attention([hidden,encoder_op])
weighted=tf.concat([decoder_embedded, context_vector],axis=2)

decoder_gru=GRU(gru_units,return_sequences=True,return_state=True)
decoder_op,h1=decoder_gru(weighted,initial_state=hidden)
dec_op=Dense(trg_vocab_size)
decoder_op=dec_op(decoder_op)

model=Model([encoder_inputs,decoder_inputs],decoder_op)

In [43]:
optimizer=keras.optimizers.RMSprop(learning_rate=0.5,clipnorm=2)

In [44]:
trg_one_hot=tf.one_hot(tar_sequences,depth=trg_vocab_size)

In [47]:
trg_one_hot.shape

TensorShape([3000, 9, 2117])

In [48]:
tar_sequences.shape

(3000, 9)

In [50]:
tar_sequences[0].shape

(9,)

In [51]:
trg_one_hot[0].shape

TensorShape([9, 2117])

In [45]:
model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

In [46]:
model.fit([src_sequences,tar_sequences],trg_one_hot,epochs=4,batch_size=64)

Epoch 1/4
47/47 [==============================] - 14s 91ms/step - loss: 13.0655 - accuracy: 0.0104
Epoch 2/4
47/47 [==============================] - 4s 93ms/step - loss: 13.0325 - accuracy: 0.0100
Epoch 3/4
47/47 [==============================] - 4s 93ms/step - loss: 12.9602 - accuracy: 0.0104
Epoch 4/4
47/47 [==============================] - 4s 92ms/step - loss: 11.8171 - accuracy: 0.3735


In [19]:
encoder_model=Model(encoder_inputs,[encoder_op,hidden])

In [52]:
encoder_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 2)]                  0         []                            
                                                                                                  
 embedding (Embedding)       (None, 2, 100)               5800      ['input_1[0][0]']             
                                                                                                  
 bidirectional (Bidirection  [(None, 2, 200),             121200    ['embedding[0][0]']           
 al)                          (None, 100),                                                        
                              (None, 100)]                                                        
                                                                                            

In [20]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 2)]                  0         []                            
                                                                                                  
 embedding (Embedding)       (None, 2, 100)               5800      ['input_1[0][0]']             
                                                                                                  
 bidirectional (Bidirection  [(None, 2, 200),             121200    ['embedding[0][0]']           
 al)                          (None, 100),                                                        
                              (None, 100)]                                                        
                                                                                              

In [51]:
model.layers[17]

In [53]:
decoder_inputs=Input(shape=(max_tar_len,))
encoder_op=Input(shape=(max_inp_len,2*gru_units))
hidden=Input(shape=(gru_units,))
decoder_embedded= model.layers[6](decoder_inputs)
context_vector=model.layers[8]([hidden,encoder_op])
context_vector=model.layers[10](context_vector,1)
context_vector=model.layers[12](context_vector,[1,tf.shape(decoder_embedded)[1],1])
encoder_op=model.layers[14](encoder_op,perm=(0,2,1))
decoder_combined=model.layers[16](context_vector,encoder_op)
weighted=model.layers[13]([decoder_embedded, context_vector],axis=2)
decoder_op,h1=model.layers[15](weighted,initial_state=hidden)
ouput=model.layers[17]([decoder_op,decoder_combined,decoder_embedded],axis=2)
dec_op=model.layers[18](ouput)

decoder_model=Model([decoder_inputs,encoder_op,hidden],[dec_op,h1])

ValueError: Found input tensor cannot be reached given provided output tensors. Please make sure the tensor KerasTensor(type_spec=TensorSpec(shape=(None, 2, 200), dtype=tf.float32, name='input_47'), name='input_47', description="created by layer 'input_47'") is included in the model inputs when building functional model.

In [32]:
decoder_model.summary()

NameError: name 'decoder_model' is not defined

In [7]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [19]:
inputs = []
outputs = []

data_file = open('spa.txt', encoding='utf-8')

count = 0
for line in data_file:
    count += 1
    if count > 3000:
        break
    if '\t' not in line:
        continue
    ip, temp_op, extra = line.rstrip().split('\t')
    op = temp_op
    inputs.append(ip)
    outputs.append(op)

In [20]:
inputs=[sentence.lower() for sentence in inputs]
outputs=[sentence.lower() for sentence in outputs]

In [21]:
outputs = ['<start> '+sentence+' <end>' for sentence in outputs]

In [22]:
EnglishTokenizer=Tokenizer(oov_token="<UNK>")
EnglishTokenizer.fit_on_texts(inputs)
inp_sequences=EnglishTokenizer.texts_to_sequences(inputs)
max_inp_len=max(len(i) for i in inp_sequences)
src_sequences=pad_sequences(inp_sequences,maxlen=max_inp_len,padding="post")
Englishword2index=EnglishTokenizer.word_index
Englishindex2word=EnglishTokenizer.index_word

In [23]:
SpanishTokenizer=Tokenizer(oov_token="<UNK>")
SpanishTokenizer.fit_on_texts(outputs)
op_sequences=SpanishTokenizer.texts_to_sequences(outputs)
max_tar_len=max(len(i) for i in op_sequences)
tar_sequences=pad_sequences(op_sequences,maxlen=max_tar_len,padding="post")
Spanishword2index=SpanishTokenizer.word_index
Spanishindex2word=SpanishTokenizer.index_word

In [24]:
src_vocab_size=len(Englishword2index)+1
trg_vocab_size=len(Spanishword2index)+1
print("src_vocab_size:",src_vocab_size)
print("tar_vocab_size:",trg_vocab_size)

src_vocab_size: 901
tar_vocab_size: 2117


In [25]:
print("max_inp_len:",max_inp_len)
print("max_tar_len:",max_tar_len)

max_inp_len: 4
max_tar_len: 9


In [26]:
gru_units=100
embed_size=100